In [217]:
import os
import requests
import pandas as pd
import json
import time
import random
from datetime import datetime, timedelta

import pymysql as mysql
import mysql.connector
import warnings
import matplotlib.pyplot as plt
import csv
from nltk.tokenize import word_tokenize

import pyodbc
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import event
from tabulate import tabulate
import plotly.express as px
import seaborn as sns
from string import punctuation
import string
import nltk
from collections import OrderedDict
from nltk.tokenize import word_tokenize


#nltk.download('punkt')
#nltk.download('stopwords')


from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.util import ngrams

from nltk.corpus import stopwords
from string import punctuation
punctuation = set(punctuation)
punctuation.update({'_', '-','‘'})


# mysql password
PASSWORD = "password"

import nbimporter
from Functions import process_text

# FDA Data

### API Request for Historical Text Documents

https://open.fda.gov/apis/other/historicaldocument/

In [8]:
# OpenFDA API endpoint for fetching historical documents
BASE_URL = 'https://api.fda.gov/other/historicaldocument.json'

# Parameters for the API request
params = {
    'limit': 1000,  # Adjust the limit as needed
    'skip': 0      # Starting point for fetching records
}

# Function to pause execution to avoid overwhelming the server
def pause_execution():
    time.sleep(5 + 10 * random.random())

# Function to fetch data from the OpenFDA API
def fetch_data(max_requests):
    documents = []
    request_count = 0  # Initialize request counter
    
    while request_count < max_requests:
        response = requests.get(BASE_URL, params=params)
        print(f"Requesting data with params: {params}")  # Debugging statement
        print(f"Response status code: {response.status_code}")  # Debugging statement

        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")  # Debugging statement
            break
        
        data = response.json()
        results = data.get('results', [])
        if not results:
            print("No more results found.")  # Debugging statement
            break

        for item in results:
            doc_type = item.get('doc_type', 'N/A')
            year = item.get('year', 'N/A')
            text = item.get('text', 'N/A')
            document = {'doc_type': doc_type, 'year': year, 'text': text}
            if document not in documents:
                documents.append(document)
        
        params['skip'] += params['limit']
        request_count += 1  # Increment request counter
        pause_execution()

    return documents

# Fetch the data
documents = fetch_data(max_requests=1)

# Save the data to a CSV file
documents_df = pd.DataFrame(documents)
documents_df.head()

Requesting data with params: {'limit': 1000, 'skip': 0}
Response status code: 200


,doc_type,year,text
0,pr,2006,FDA NEWS RELEASE\nFOR IMMEDIATE RELEASE\n\nFeb...
1,pr,2006,FDA NEWS RELEASE\nFOR IMMEDIATE RELEASE\n\nFeb...
2,talk,1991,I FOOD _-AND DRUG ADMINISTRATION ‘ 'i\nU. S. D...
3,pr,1934,iNFORMATION FOR THE PRESS\n\nu 8. DEPARTMENT O...
4,pr,1975,\n\nU. S. DEPARTMENT OF HEALTH. EDUCATION. AN...


In [9]:
documents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   doc_type  1000 non-null   object
 1   year      1000 non-null   int64 
 2   text      1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [20]:
# Save csv file to data library
#csv_file_path = 'DataLibrary/raw_documents.csv'
#documents_df.to_csv(csv_file_path, index=False)

# Save the data to a JSON file
#json_file_path = 'DataLibrary/raw_documents.json'
#with open(json_file_path, 'w') as json_file:
#    json.dump(documents, json_file, indent=4)

#print(f"Data saved to {csv_file_path} and {json_file_path}")

### API Request for Adverse Events Data

https://open.fda.gov/apis/drug/event/

In [10]:
# OpenFDA API endpoint for fetching drug event data
BASE_URL = 'https://api.fda.gov/drug/event.json'

# Parameters for the API request
params = {
    'limit': 1000,  # Maximum limit per request
    'skip': 0      # Starting point for fetching records
}

# Function to pause execution to avoid overwhelming the server
def pause_execution():
    time.sleep(5 + 10 * random.random())

# Function to fetch data from the OpenFDA API
def fetch_data(max_requests):
    events = []
    request_count = 0  # Initialize request counter
    
    while request_count < max_requests:
        response = requests.get(BASE_URL, params=params)
        print(f"Requesting data with params: {params}")  # Debugging statement
        print(f"Response status code: {response.status_code}")  # Debugging statement

        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")  # Debugging statement
            break
        
        data = response.json()
        results = data.get('results', [])
        if not results:
            print("No more results found.")  # Debugging statement
            break

        for item in results:
            events.append(item)
        
        params['skip'] += params['limit']
        request_count += 1  # Increment request counter
        pause_execution()

    return events

# Fetch the data
events = fetch_data(max_requests=2)

# Convert the data to a pandas DataFrame
events_df = pd.json_normalize(events)

# Display the first few rows of the DataFrame
events_df.head()

Requesting data with params: {'limit': 1000, 'skip': 0}
Response status code: 200
Requesting data with params: {'limit': 1000, 'skip': 1000}
Response status code: 200


,safetyreportid,transmissiondateformat,transmissiondate,serious,seriousnessdeath,receivedateformat,receivedate,receiptdateformat,receiptdate,fulfillexpeditecriteria,...,seriousnessother,occurcountry,patient.patientagegroup,seriousnesshospitalization,patient.summary.narrativeincludeclinical,seriousnesslifethreatening,patient.patientweight,primarysource.literaturereference,seriousnesscongenitalanomali,authoritynumb
0,5801206-7,102,20090109,1,1,102,20080707,102,20080625,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10003300,102,20141002,1,NaN,102,20140306,102,20140306,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10003301,102,20141002,1,NaN,102,20140228,102,20140228,2,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10003302,102,20141002,2,NaN,102,20140312,102,20140312,2,...,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10003304,102,20141212,2,NaN,102,20140312,102,20140424,2,...,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 42 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   safetyreportid                               2000 non-null   object 
 1   transmissiondateformat                       2000 non-null   object 
 2   transmissiondate                             2000 non-null   object 
 3   serious                                      2000 non-null   object 
 4   seriousnessdeath                             145 non-null    object 
 5   receivedateformat                            2000 non-null   object 
 6   receivedate                                  2000 non-null   object 
 7   receiptdateformat                            2000 non-null   object 
 8   receiptdate                                  2000 non-null   object 
 9   fulfillexpeditecriteria                      2000 non-null   object 
 10  

In [23]:
# Save the data to a JSON file
#json_file_path = 'DataLibrary/raw_events.json'
#with open(json_file_path, 'w') as json_file:
#    json.dump(events, json_file, indent=4)

# Save the data to a CSV file
#csv_file_path = 'DataLibrary/raw_events.csv'
#events_df.to_csv(csv_file_path, index=False)

#print(f"Data saved to {csv_file_path} and {json_file_path}")

### API Request for Drug Labels

https://open.fda.gov/apis/drug/label/

In [173]:
# OpenFDA API endpoint for fetching drug label data
BASE_URL = 'https://api.fda.gov/drug/label.json'

# Parameters for the API request
params = {
    'limit': 1000,  # Maximum limit per request
    'skip': 0      # Starting point for fetching records
}

# Function to pause execution to avoid overwhelming the server
def pause_execution():
    time.sleep(5 + 10 * random.random())

# Function to fetch data from the OpenFDA API
def fetch_data(max_requests):
    labels = []
    request_count = 0  # Initialize request counter
    
    while request_count < max_requests:
        response = requests.get(BASE_URL, params=params)
        print(f"Requesting data with params: {params}")  # Debugging statement
        print(f"Response status code: {response.status_code}")  # Debugging statement

        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")  # Debugging statement
            break
        
        data = response.json()
        results = data.get('results', [])
        if not results:
            print("No more results found.")  # Debugging statement
            break

        for item in results:
            labels.append(item)
        
        params['skip'] += params['limit']
        request_count += 1  # Increment request counter
        pause_execution()

    return labels

# Fetch the data
labels = fetch_data(max_requests=2)

# Convert the data to a pandas DataFrame
labels_df = pd.json_normalize(labels)

# Display the first few rows of the DataFrame
labels_df.head()

Requesting data with params: {'limit': 1000, 'skip': 0}
Response status code: 200
Requesting data with params: {'limit': 1000, 'skip': 1000}
Response status code: 200


,effective_time,inactive_ingredient,purpose,keep_out_of_reach_of_children,warnings,questions,spl_product_data_elements,version,dosage_and_administration,pregnancy_or_breast_feeding,...,components,intended_use_of_the_device,mechanism_of_action_table,general_precautions_table,drug_and_or_laboratory_test_interactions_table,accessories,ask_doctor_table,when_using_table,ask_doctor_or_pharmacist_table,do_not_use_table
0,20210902,[INACTIVE INGREDIENTS Sucrose],"[USES USES: Temporary Relief - Acne, Boils* * ...",[Keep this and all medication out of reach of ...,[WARNINGS This product is to be used for self-...,[QUESTIONS OR COMMENTS www.Rxhomeo.com | 1.888...,[SILICEA SILICEA SUCROSE SILICON DIOXIDE SILIC...,2,"[DOSAGE Adults- Take 4 or 6 Pellets by mouth, ...","[As with any drug, if you are pregnant, or nur...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20150109,"[INGREDIENTS: TALC, POLYMETHYL METHACRYLATE, V...",[Purpose Sunscreen],[Keep out of reach of children If product is s...,[Warnings For external use only.],NaN,[CHANTECAILLE PROTECTION NATURELLE BRONZE SPF ...,4,[Directions Protection Naturelle SPF 46 PA+++ ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20230802,[INACTIVE INGREDIENTS Sucrose/Lactose],[USES To relieve the symptoms of itching.],[KEEP OUT OF REACH OF CHILDREN Keep this and a...,[STOP USE AND ASK DOCTOR If symptoms persist/w...,NaN,[Mezereum DAPHNE MEZEREUM BARK SUCROSE LACTOSE...,3,[DIRECTIONS Adults: Dissolve 3 to 5 under the ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20230905,NaN,NaN,NaN,[WARNINGS NOT FOR INJECTION. Ofloxacin ophthal...,NaN,[Ofloxacin Ofloxacin OFLOXACIN OFLOXACIN Sodiu...,7,[DOSAGE AND ADMINISTRATION The recommended dos...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230403,NaN,NaN,NaN,NaN,NaN,[Naproxen Naproxen NAPROXEN NAPROXEN CROSCARME...,27,[2 DOSAGE AND ADMINISTRATION Use the lowest ef...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pd.set_option('display.max_info_columns', 140)
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 140 columns):
 #    Column                                                            Non-Null Count  Dtype 
---   ------                                                            --------------  ----- 
 0    effective_time                                                    2000 non-null   object
 1    inactive_ingredient                                               1217 non-null   object
 2    purpose                                                           1189 non-null   object
 3    keep_out_of_reach_of_children                                     1185 non-null   object
 4    warnings                                                          1587 non-null   object
 5    questions                                                         657 non-null    object
 6    spl_product_data_elements                                         1999 non-null   object
 7    version                        

In [26]:
# Save the data to a JSON file
#json_file_path = 'DataLibrary/raw_labels.json'
#with open(json_file_path, 'w') as json_file:
#    json.dump(labels, json_file, indent=4)

# Save the data to a CSV file
#csv_file_path = 'DataLibrary/raw_labels.csv'
#labels_df.to_csv(csv_file_path, index=False)

#print(f"Data saved to {csv_file_path} and {json_file_path}")

# National Library of Medicine - Standardized Drug Names and Information (maybe not needed)

https://lhncbc.nlm.nih.gov/RxNav/APIs/RxNormAPIs.html

### API Request for RxNorm

In [27]:
# Base URL for RxTerms API to get all drug names
#BASE_URL = 'https://rxnav.nlm.nih.gov/REST/RxTerms/rxcui'

# Function to pause execution to avoid overwhelming the server
#def pause_execution():
#    time.sleep(1 + 2 * random.random())

# Function to fetch data from the RxTerms API
#def fetch_data(max_requests):
#    drugs = []
#    start_rxcui = 1  # Starting RxCUI for the search
#    step = 500      # Step size to iterate through RxCUIs
#    request_count = 0  # Initialize request counter

#    while request_count < max_requests:
#        batch_drugs = []
#        for i in range(start_rxcui, start_rxcui + step):
#            url = f"{BASE_URL}/{i}/allinfo.json"
#            response = requests.get(url)

#            if response.status_code != 200:
#                print(f"Failed to fetch data: {response.status_code}")  # Debugging statement
#                continue

#            if response.content.strip():  # Check if the response content is not empty
#                data = response.json()
#                properties = data.get('rxtermsProperties', {})

#                if properties:
#                    if properties.get('marketed', '').lower() == 'true':
#                        properties['rxcui'] = i
#                        batch_drugs.append(properties)

#        drugs.extend(batch_drugs)  # Add the batch results to the main list
#        print(f"Processed batch starting with RxCUI: {start_rxcui}")  # Debugging statement for each batch

        # Update the start_rxcui for the next batch
 #       start_rxcui += step
#      request_count += 1  # Increment request counter

        # Break condition to stop if no more data in the batch
#        if not batch_drugs:
#            break

#        pause_execution()

#    return drugs

# Fetch the data
#drugs = fetch_data(max_requests=1)

# Convert the data to a pandas DataFrame
#drugs_df = pd.DataFrame(drugs)

# Display the first few rows of the DataFrame
#drugs_df.head()

Processed batch starting with RxCUI: 1


""


In [29]:
# Save the data to a JSON file
#json_file_path = 'DataLibrary/rxterms_drugs.json'
#with open(json_file_path, 'w') as json_file:
#    json.dump(drugs, json_file, indent=4)

# Save the data to a CSV file
#csv_file_path = 'DataLibrary/rxterms_drugs.csv'
#drugs_df.to_csv(csv_file_path, index=False)

#print(f"Data saved to {csv_file_path} and {json_file_path}")

# Medicaid Drug Prices - 2024 Report - API 

https://data.medicaid.gov/dataset/99315a95-37ac-4eee-946a-3c523b4c481e#data-table

In [218]:
# Base URL for the Medicaid API for NADAC data
BASE_URL = 'https://data.medicaid.gov/api/1/datastore/sql'

# Initial SQL query to fetch data with a limit of 500 and initial offset 0
# Queries 2024 Database
query_template = '[SELECT * FROM f3b5cf6b-07cc-5f75-8d7b-0a1090b3f7e9][LIMIT 500 OFFSET {}]'

# Function to pause execution to avoid overwhelming the server
def pause_execution():
    # Pause execution for a random time between 1 and 3 seconds to avoid hitting the server too hard
    time.sleep(1 + 2 * random.random())

# Function to fetch data from the Medicaid API in batches
def fetch_nadac_data():
    all_data = []  # Initialize an empty list to store all fetched data
    offset = 0  # Starting offset for pagination
    max_requests = 2  # Maximum number of requests to fetch 1000 records
    
    for _ in range(max_requests):  # Loop to fetch data in batches
        # Modify the query to include the current offset for pagination
        query = query_template.format(offset)
        params = {'query': query}  # Set the parameters for the API request

        response = requests.get(BASE_URL, params=params)  # Send the request to the API
        print(f"Requesting data with offset: {offset}")  # Debugging statement to show current offset
        print(f"Response status code: {response.status_code}")  # Debugging statement to show response status

        if response.status_code != 200:  # Check if the request was unsuccessful
            print(f"Failed to fetch data: {response.status_code}")  # Debugging statement for error status
            try:
                error_details = response.json()
                print(f"Error message: {error_details.get('message', 'No message provided')}")  # Print the error message
                print(f"Error details: {error_details.get('data', 'No additional data provided')}")  # Print additional error details
            except ValueError:
                print("Failed to decode JSON error response.")  # Debugging statement for JSON error
            break  # Exit the loop if there was an error

        try:
            data = response.json()  # Parse the response JSON
            #print(f"Raw response data: {data}")  # Debugging statement to show raw response data
            
            if not isinstance(data, list):  # Check if the response is an array
                print("Expected a list but got a different structure.")
                break

            if not data:  # Check if the list is empty
                print("No more results found.")  # Debugging statement to indicate no more data
                break  # Exit the loop if no more data is found

            all_data.extend(data)  # Append the fetched data to the main list
            offset += len(data)  # Increment the offset by the number of records fetched
            print(f"Fetched {len(data)} records. Total so far: {len(all_data)}")  # Debugging statement to show fetched data count
        except ValueError:  # Handle any JSON decoding errors
            print("Failed to decode JSON response.")  # Debugging statement for JSON error
            break  # Exit the loop if there was an error decoding JSON
        
        pause_execution()  # Pause before the next request to avoid overwhelming the server

    return all_data  # Return the collected data

# Fetch the data
prices = fetch_nadac_data()

# Convert the data to a pandas DataFrame
prices_df = pd.DataFrame(prices)

# Display the first few rows of the DataFrame
prices_df.head()
print(f"Total records fetched: {len(prices_df)}")  # Debugging statement

Requesting data with offset: 0
Response status code: 200
Fetched 500 records. Total so far: 500
Requesting data with offset: 500
Response status code: 200
Fetched 500 records. Total so far: 1000
Total records fetched: 1000


In [15]:
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   NDC Description                            1000 non-null   object
 1   NDC                                        1000 non-null   object
 2   NADAC_Per_Unit                             1000 non-null   object
 3   Effective Date                             1000 non-null   object
 4   Pricing_Unit                               1000 non-null   object
 5   Pharmacy_Type_Indicator                    1000 non-null   object
 6   OTC                                        1000 non-null   object
 7   Explanation_Code                           1000 non-null   object
 8   Classification_for_Rate_Setting            1000 non-null   object
 9   Corresponding_Generic_Drug_NADAC_Per_Unit  1000 non-null   object
 10  Corresponding_Generic_Drug_Effective_

In [175]:
prices_df.head()

,NDC Description,NDC,NADAC_Per_Unit,Effective Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date
0,12HR NASAL DECONGEST ER 120 MG,24385005452,0.28255,2022-12-21,EA,C/I,Y,1,G,,,2023-01-04
1,12HR NASAL DECONGEST ER 120 MG,70000047501,0.28255,2022-12-21,EA,C/I,Y,1,G,,,2023-01-04
2,24H NASAL ALLERGY 55 MCG SPRAY,46122038576,0.78326,2022-12-21,ML,C/I,Y,"1, 5",G,,,2023-01-04
3,24HR ALLERGY(LEVOCETIRZN) 5 MG,70000036201,0.15365,2022-12-21,EA,C/I,Y,1,G,,,2023-01-04
4,24HR ALLERGY(LEVOCETIRZN) 5 MG,70000036202,0.15365,2022-12-21,EA,C/I,Y,1,G,,,2023-01-04


# Select Variables of Interest from Datasets

In [219]:
#documents_df = pd.read_csv('DataLibrary/raw_documents.csv')
documents_table = pd.DataFrame(documents_df, 
                               columns = ["doc_type",  # pr: press release, talk: speech  
                                          "year", 
                                          "text"])


#events_df = pd.read_csv('DataLibrary/raw_events.csv.csv')
events_table = pd.DataFrame(events_df, 
                            columns = ["safetyreportid", #case report number
                                        "transmissiondate", # date the record was created
                                        "serious", # seriousness - 1 = result in death, lifethreatening,hospital,etc. 2 = not in the above
                                        "seriousnessdeath", # 1 = death
                                        #  "receivedateformat", #always YYYYMMDD
                                        #  "receivedate", #date report received by FDA
                                        "primarysource.reportercountry", # country of the reporter of this event
                                        "patient.patientsetage", # age of patient when event occured
                                        "patient.patientsex", # sex of patient
                                        "patient.reaction", #DICTIONARY LIST - fix
                                        "patient.drug", #DICTIONARY LIST - fix
                                        # "patient.patientagegroup", #1 = neonate, 2 = infant, 3=child, 4=adolescent, 5=adult, 6=elderly
                                        # "patient.patientweight", # weight in kg
                                        "seriousnesscongentialanomali", # 1 if congenital anomaly (fetal abnormality),absent otherwise
                                        # "authoritynumb", #regulatory authority case #
                                        "reportduplicate"])

#labels_df = pd.read_csv('')

labels_table = pd.DataFrame(labels_df, 
                            columns = ["effective_time",
                                        #"inactive_ingredient",  # Included in spl_product_data_elements
                                       "purpose",
                                       "warnings",  # Combine text with contraindications and precautions
                                        #"questions",  # Too much missing data
                                       "spl_product_data_elements",  # Contains both active and inactive ingredients
                                        #"version",  # Label version, not needed since ndc code captures this information
                                        #"package_label_principal_display_panel",  # Package label text, redundant information
                                        #"active_ingredient",  # Included in spl_product_data_elements
                                        "openfda.brand_name",  # Combine into list with generic
                                        "openfda.generic_name",  # Combine into list with brand
                                        "openfda.manufacturer_name",  # Company that produces the drug
                                        "openfda.product_type",  # Denotes whether OTC and prescription, and human or animal
                                        "openfda.substance_name",  # Combine with brand and generic
                                        "openfda.product_ndc",  # Do NOT tokenize, NDC Product code, use as database key, already a list
                                        #"openfda.package_ndc",  # Product NDC code plus one extra number for package type, Redundant
                                        "openfda.rxcui",  # Do NOT tokenize, normed drug code number, retain for database keys
                                        "contraindications",  # Combine with warnings and precautions
                                        "precautions"  # Combine with warnings and contraindications
                                        ])

#drugs_table = pd.read_csv('')

prices_table = pd.DataFrame(prices_df, 
                            columns = [#"NDC Description",  # Identifies the drug name/dosage, Redundant Information with labels_table
                                       "NDC",  # 11 character FDA code, Do NOT Tokenize, database key
                                       "NADAC_Per_Unit",  #  The National Average Drug Acquisition Cost per unit, do NOT tokenize
                                       "Effective Date",  # The effective date of the NADAC Per Unit cost
                                        #"Pricing_Unit",  # Pricing unit for NDC ('ML', 'GM 'or 'EA'), Not necessary, just use price
                                        #"Pharmacy_Type_Indicator",  # 'C/I' data from Chain/Independent pharmacies, does not provide helpful info
                                        #"OTC",  # Indicates whether NDC is over-the-counter (OTC), redundant with labels_table
                                         #"Explanation Code",  # NADAC Calculation Method, not necessary for this project, too much missing data
                                                                       # Code 1: Inform from most recent survey.
                                                                       # Code 2: Cost within ± 2%, carried forward from previous
                                                                       # Code 3: Published pricing, or help desk inquiry
                                                                       # Code 4: Carried forward from previous year
                                                                       # Code 5: Based on package size.
                                                                       # Code 6: CMS Covered Outpatient Drug Product File drug category 
                                        "Classification_for_Rate_Setting",  # B: brand, 
                                                                            # G: generic
                                                                            # B-ANDA: brand under ANDA
                                                                            # B-BIO: biosimilar brand  
                                        "Corresponding_Generic_Drug_NADAC_Per_Unit",
                                        "Corresponding_Generic_Drug_Effective_Date"
                                        #"As of Date"
                                      ])
                                            



# Preprocess Documents Table

In [36]:
start_time = time.time()

documents_table = pd.DataFrame(documents_df, columns=["doc_type", "year", "text"])

documents_table = add_sequential_index(documents_table,'doc_id')

documents_table.info()

print(documents_table['doc_type'].unique())

documents_table['text_processed'] = documents_table['text'].apply(lambda x: process_text(x))

end_time = time.time()

duration = end_time - start_time

print("Time taken for", len(documents_table), "records: ", round(duration,2), " seconds")

documents_table.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   doc_id    1000 non-null   int64 
 1   doc_type  1000 non-null   object
 2   year      1000 non-null   int64 
 3   text      1000 non-null   object
dtypes: int64(2), object(2)
memory usage: 31.4+ KB
['pr' 'talk']
Time taken for 1000 records:  4.57  seconds


,doc_id,doc_type,year,text,text_processed
0,1,pr,2006,FDA NEWS RELEASE\nFOR IMMEDIATE RELEASE\n\nFeb...,"[fda, news, release, immediate, release, febru..."
1,2,pr,2006,FDA NEWS RELEASE\nFOR IMMEDIATE RELEASE\n\nFeb...,"[fda, news, release, immediate, release, febru..."
2,3,talk,1991,I FOOD _-AND DRUG ADMINISTRATION ‘ 'i\nU. S. D...,"[food, drug, administration, dc, partmcnt, hea..."
3,4,pr,1934,iNFORMATION FOR THE PRESS\n\nu 8. DEPARTMENT O...,"[information, press, u, 8, department, agricul..."
4,5,pr,1975,\n\nU. S. DEPARTMENT OF HEALTH. EDUCATION. AN...,"[department, health, education, welfare, food,..."


# PreProcess Events Table

In [40]:
len(documents_table)

events_table.info()

duplicates = events_table.duplicated(subset=['safetyreportid']).sum()

print("Duplicates: ", duplicates) 

events_table['transmissiondate'] = pd.to_datetime(events_table['transmissiondate'], format='%Y%m%d')

# Save 1 as life threatening, hospitalization, disability, congenital anomaly, or serious condition, 0 as not that
events_table['serious'] = events_table['serious'].replace({1: 0, 2: 1}) 


events_table.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   safetyreportid                 2000 non-null   object        
 1   transmissiondate               2000 non-null   datetime64[ns]
 2   serious                        2000 non-null   object        
 3   seriousnessdeath               145 non-null    object        
 4   primarysource.reportercountry  2000 non-null   object        
 5   patient.patientsetage          0 non-null      float64       
 6   patient.patientsex             1988 non-null   object        
 7   patient.reaction               2000 non-null   object        
 8   patient.drug                   2000 non-null   object        
 9   seriousnesscongentialanomali   0 non-null      float64       
 10  reportduplicate                0 non-null      float64       
dtypes: datetime64[ns]

,safetyreportid,transmissiondate,serious,seriousnessdeath,primarysource.reportercountry,patient.patientsetage,patient.patientsex,patient.reaction,patient.drug,seriousnesscongentialanomali,reportduplicate
0,5801206-7,2009-01-09,1,1,CANADA,NaN,1,[{'reactionmeddrapt': 'DRUG ADMINISTRATION ERR...,"[{'drugcharacterization': '1', 'medicinalprodu...",NaN,NaN
1,10003300,2014-10-02,1,NaN,US,NaN,2,"[{'reactionmeddraversionpt': '17.0', 'reaction...","[{'drugcharacterization': '1', 'medicinalprodu...",NaN,NaN
2,10003301,2014-10-02,1,NaN,US,NaN,2,"[{'reactionmeddraversionpt': '17.0', 'reaction...","[{'drugcharacterization': '1', 'medicinalprodu...",NaN,NaN
3,10003302,2014-10-02,2,NaN,US,NaN,1,"[{'reactionmeddraversionpt': '17.0', 'reaction...","[{'drugcharacterization': '1', 'medicinalprodu...",NaN,NaN
4,10003304,2014-12-12,2,NaN,US,NaN,2,"[{'reactionmeddraversionpt': '17.0', 'reaction...","[{'drugcharacterization': '1', 'medicinalprodu...",NaN,NaN


# PreProcess Labels Table

In [177]:
labels_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   effective_time             2000 non-null   object
 1   purpose                    1189 non-null   object
 2   warnings                   1587 non-null   object
 3   spl_product_data_elements  1999 non-null   object
 4   openfda.brand_name         984 non-null    object
 5   openfda.generic_name       984 non-null    object
 6   openfda.manufacturer_name  984 non-null    object
 7   openfda.product_type       984 non-null    object
 8   openfda.substance_name     974 non-null    object
 9   openfda.product_ndc        984 non-null    object
 10  openfda.rxcui              725 non-null    object
 11  contraindications          753 non-null    object
 12  precautions                423 non-null    object
dtypes: object(13)
memory usage: 203.2+ KB


In [179]:
# Define text columns
object_columns = labels_table.select_dtypes(include=['object']).columns

# Exclude columns that should not be tokenized, index codes
object_columns = object_columns.drop(['openfda.product_ndc', 'openfda.rxcui'], errors='ignore')
labels_table[object_columns] = labels_table[object_columns].astype(str)

# Apply text processing to the selected text columns and add new processed columns to dataframe
for col in object_columns:
    new_col_name = col + '_processed'
    labels_table[new_col_name] = labels_table[col].apply(process_label_text)

In [181]:
labels_table.head(2)

,effective_time,purpose,warnings,spl_product_data_elements,openfda.brand_name,openfda.generic_name,openfda.manufacturer_name,openfda.product_type,openfda.substance_name,openfda.product_ndc,...,purpose_processed,warnings_processed,spl_product_data_elements_processed,openfda.brand_name_processed,openfda.generic_name_processed,openfda.manufacturer_name_processed,openfda.product_type_processed,openfda.substance_name_processed,contraindications_processed,precautions_processed
0,20210902,"['USES USES: Temporary Relief - Acne, Boils* *...",['WARNINGS This product is to be used for self...,['SILICEA SILICEA SUCROSE SILICON DIOXIDE SILI...,['SILICEA'],['SILICEA'],"['Rxhomeo Private Limited d.b.a. Rxhomeo, Inc']",['HUMAN OTC DRUG'],['SILICON DIOXIDE'],[15631-0404],...,"[us, temporary, relief, acne, boil, claim, bas...","[warning, product, used, self, limiting, condi...","[silicea, sucrose, silicon, dioxide]",[silicea],[silicea],"[rxhomeo, private, limited, b, inc]","[human, otc, drug]","[silicon, dioxide]",[nan],[nan]
1,20150109,['Purpose Sunscreen'],['Warnings For external use only.'],['CHANTECAILLE PROTECTION NATURELLE BRONZE SPF...,nan,nan,nan,nan,nan,NaN,...,"[purpose, sunscreen]","[warning, external, use]","[chantecaille, protection, naturelle, bronze, ...",[nan],[nan],[nan],[nan],[nan],[nan],[nan]


### Examine all variables for quality, content, and redundancy

In [182]:
labels_table['spl_product_data_elements_processed'].head()

0                 [silicea, sucrose, silicon, dioxide]
1    [chantecaille, protection, naturelle, bronze, ...
2    [mezereum, daphne, bark, sucrose, lactose, white]
3    [ofloxacin, sodium, chloride, hydrochloric, ac...
4    [naproxen, croscarmellose, sodium, povidone, u...
Name: spl_product_data_elements_processed, dtype: object

In [185]:
labels_table['purpose'].head()

0    ['USES USES: Temporary Relief - Acne, Boils* *...
1                                ['Purpose Sunscreen']
2         ['USES To relieve the symptoms of itching.']
3                                                  nan
4                                                  nan
Name: purpose, dtype: object

In [186]:
labels_table['purpose_processed'].head()

0    [us, temporary, relief, acne, boil, claim, bas...
1                                 [purpose, sunscreen]
2                      [us, relieve, symptom, itching]
3                                                [nan]
4                                                [nan]
Name: purpose_processed, dtype: object

In [187]:
labels_table['warnings'].head()

0    ['WARNINGS This product is to be used for self...
1                  ['Warnings For external use only.']
2    ['STOP USE AND ASK DOCTOR If symptoms persist/...
3    ['WARNINGS NOT FOR INJECTION. Ofloxacin ophtha...
4                                                  nan
Name: warnings, dtype: object

In [188]:
labels_table['warnings_processed'].head()

0    [warning, product, used, self, limiting, condi...
1                             [warning, external, use]
2    [stop, use, ask, doctor, symptom, persist, wor...
3    [warning, injection, ofloxacin, ophthalmic, so...
4                                                [nan]
Name: warnings_processed, dtype: object

In [189]:
labels_table['contraindications'].head()

0                                                  nan
1                                                  nan
2                                                  nan
3    ['CONTRAINDICATIONS Ofloxacin ophthalmic solut...
4    ['4 CONTRAINDICATIONS Naproxen tablets and nap...
Name: contraindications, dtype: object

In [190]:
labels_table['contraindications_processed'].head()

0                                                [nan]
1                                                [nan]
2                                                [nan]
3    [contraindication, ofloxacin, ophthalmic, solu...
4    [4, contraindication, naproxen, tablet, sodium...
Name: contraindications_processed, dtype: object

In [191]:
labels_table['precautions'].head()

0                                                  nan
1                                                  nan
2                                                  nan
3    ['PRECAUTIONS General: As with other anti-infe...
4                                                  nan
Name: precautions, dtype: object

In [192]:
labels_table['precautions_processed'].head()

0                                                [nan]
1                                                [nan]
2                                                [nan]
3    [precaution, general, anti, infectives, prolon...
4                                                [nan]
Name: precautions_processed, dtype: object

In [194]:
labels_table['openfda.brand_name_processed'].head()

0     [silicea]
1         [nan]
2         [nan]
3         [nan]
4    [naproxen]
Name: openfda.brand_name_processed, dtype: object

In [195]:
labels_table['openfda.generic_name_processed'].head()

0     [silicea]
1         [nan]
2         [nan]
3         [nan]
4    [naproxen]
Name: openfda.generic_name_processed, dtype: object

In [196]:
labels_table['openfda.manufacturer_name_processed'].head()

0    [rxhomeo, private, limited, b, inc]
1                                  [nan]
2                                  [nan]
3                                  [nan]
4                 [medication, solution]
Name: openfda.manufacturer_name_processed, dtype: object

In [198]:
labels_table['openfda.product_type_processed'].head()

0             [human, otc, drug]
1                          [nan]
2                          [nan]
3                          [nan]
4    [human, prescription, drug]
Name: openfda.product_type_processed, dtype: object

In [199]:
labels_table['openfda.substance_name_processed'].head()

0    [silicon, dioxide]
1                 [nan]
2                 [nan]
3                 [nan]
4            [naproxen]
Name: openfda.substance_name_processed, dtype: object

In [200]:
labels_table['openfda.product_ndc'].head()

0    [15631-0404]
1             NaN
2             NaN
3             NaN
4    [50090-0481]
Name: openfda.product_ndc, dtype: object

### Feature Creation for Labels Data

In [203]:
# Reformat effective time as date
labels_table['effective_time'] = pd.to_datetime(labels_table['effective_time'], format='%Y%m%d')

# Combine warnings, contraindications, and precautions text
labels_table['all_warnings_processed'] = labels_table['warnings_processed'] + labels_table['contraindications_processed'] + labels_table['precautions_processed']
# Remove duplicate tokens from combination list
labels_table['all_warnings_processed'] = labels_table['all_warnings_processed'].apply(remove_duplicates)
 

# Combine brand name, generic name, and substance name
labels_table['all_name_processed'] = labels_table['openfda.brand_name_processed'] + labels_table['openfda.generic_name_processed'] + labels_table['openfda.substance_name_processed']
# Remove duplicate tokens to retain unique descriptors only
labels_table['all_name_processed'] = labels_table['all_name_processed'].apply(remove_duplicates)

# Create a category variable from the "openfda.product_type"
labels_table['category'] = labels_table['openfda.product_type_processed'].apply(classify_product_type)


### Final List of Features for Labels Data

In [205]:
labels_table = labels_table[['openfda.rxcui', 'openfda.product_ndc', 'effective_time',
                            'spl_product_data_elements_processed', 'all_warnings_processed', 'all_name_processed',
                            'category', 'openfda.manufacturer_name_processed',
                            'purpose_processed']]

In [206]:
labels_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   openfda.rxcui                        725 non-null    object        
 1   openfda.product_ndc                  984 non-null    object        
 2   effective_time                       2000 non-null   datetime64[ns]
 3   spl_product_data_elements_processed  2000 non-null   object        
 4   all_warnings_processed               2000 non-null   object        
 5   all_name_processed                   2000 non-null   object        
 6   openfda.product_type_processed       2000 non-null   object        
 7   openfda.manufacturer_name_processed  2000 non-null   object        
 8   purpose_processed                    2000 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 140.8+ KB


# PreProcess Drugs Table

# PreProcess Prices Table

### Feature Engineering 

In [220]:
# Reformat dates as dates
prices_table['Effective Date'] = pd.to_datetime(prices_table['Effective Date'])
prices_table['Corresponding_Generic_Drug_Effective_Date'] = pd.to_datetime(prices_table['Corresponding_Generic_Drug_Effective_Date'])

In [221]:
prices_table['Classification_for_Rate_Setting'].value_counts()

Classification_for_Rate_Setting
G         932
B          62
B-ANDA      6
Name: count, dtype: int64

In [222]:
# Update classification to binary generic/brand, G:1 B:0
prices_table['Classification_for_Rate_Setting'] = prices_table['Classification_for_Rate_Setting'].apply(lambda x: 1 if x == 'G' else 0)

In [104]:
prices_table.head()

,NDC Description,NDC,NADAC_Per_Unit,Effective Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date
0,12HR NASAL DECONGEST ER 120 MG,24385005452,0.28255,2022-12-21,EA,1,Y,NaN,G,,,2023-01-04
1,12HR NASAL DECONGEST ER 120 MG,70000047501,0.28255,2022-12-21,EA,1,Y,NaN,G,,,2023-01-04
2,24H NASAL ALLERGY 55 MCG SPRAY,46122038576,0.78326,2022-12-21,ML,1,Y,NaN,G,,,2023-01-04
3,24HR ALLERGY(LEVOCETIRZN) 5 MG,70000036201,0.15365,2022-12-21,EA,1,Y,NaN,G,,,2023-01-04
4,24HR ALLERGY(LEVOCETIRZN) 5 MG,70000036202,0.15365,2022-12-21,EA,1,Y,NaN,G,,,2023-01-04


### Final list of features for prices data

In [223]:
prices_table = prices_table[['NDC', 'NADAC_Per_Unit', 'Effective Date', 'Classification_for_Rate_Setting', 
                             'Corresponding_Generic_Drug_NADAC_Per_Unit', 'Corresponding_Generic_Drug_Effective_Date']]

# Standardize/Simplify Variable Name Format Across Tables

In [210]:
labels_table = labels_table.rename(columns = {'openfda.rxcui': 'rxcui',
                         'openfda.product_ndc': 'ndc',
                         'effective_time': 'effect_date',
                         'spl_product_data_elements_processed': 'ingredients',
                         'all_warnings_processed': 'warnings',
                         'all_name_processed': 'drug_names',
                         'openfda.product_type_processed': 'product_type',
                         'openfda.manufacturer_name_processed': 'manufacturer',
                         'purpose_processed': 'purpose'})

In [211]:
labels_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   rxcui         725 non-null    object        
 1   ndc           984 non-null    object        
 2   effect_date   2000 non-null   datetime64[ns]
 3   ingredients   2000 non-null   object        
 4   warnings      2000 non-null   object        
 5   drug_names    2000 non-null   object        
 6   product_type  2000 non-null   object        
 7   manufacturer  2000 non-null   object        
 8   purpose       2000 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 140.8+ KB


In [225]:
prices_table = prices_table.rename(columns = {'NDC': 'ndc',
                         'NADAC_Per_Unit': 'unit_price',
                         'Effective Date': 'effect_date',
                         'Classification_for_Rate_Setting': 'generic_brand',
                         'Corresponding_Generic_Drug_NADAC_Per_Unit': 'generic_price',
                         'Corresponding_Generic_Drug_Effective_Date': 'generic_date'})

In [226]:
prices_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ndc            1000 non-null   object        
 1   unit_price     1000 non-null   object        
 2   effect_date    1000 non-null   datetime64[ns]
 3   generic_brand  1000 non-null   int64         
 4   generic_price  1000 non-null   object        
 5   generic_date   29 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 47.0+ KB


# Combining Data into SQL Database Based on Keys

In [6]:
# Keys in FDA data = rxcui OR package_ndc
# Key in medicaid data = NDC
# Key in RxNorm data = rxcui

### Create new database to store data

In [ ]:
# connect to MySQL server
connection = mysql.connector.connect(host="localhost", user="root", password=PASSWORD)

# Create a cursor object
cursor = connection.cursor()

# Drop the database if it exists - this is so we can start fresh (at least while developing)
cursor.execute("DROP DATABASE IF EXISTS pharma_db")

# Create the 'covid_db' database
cursor.execute("CREATE DATABASE pharma_db")

# Switch to the 'covid_db' database
cursor.execute("USE pharma_db")

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

### Create Read-Only User Access for Public

In [ ]:
# public username and password

### Create Edit Privelages for Us

In [ ]:
# private username and password

### Create dataframes from API request dataframes

In [ ]:
# Connect to the MySQL server
connection = mysql.connector.connect(
    host="localhost", user="root", password=PASSWORD, database="pharma_db"
)

# Create a cursor object
cursor = connection.cursor()

# Create the table
create_table1_query = """
CREATE TABLE IF NOT EXISTS `table_name` (
  vars
) ENGINE=InnoDB;
"""
cursor.execute(create_table1_query)


# Create the table
create_table2_query = """
CREATE TABLE IF NOT EXISTS `table_name` (
  vars
) ENGINE=InnoDB;
"""
cursor.execute(create_table2_query)

# Create the table
create_table3_query = """
CREATE TABLE IF NOT EXISTS `table_name` (
  vars
) ENGINE=InnoDB;
"""
cursor.execute(create_table3_query)

# Create the table
create_table4_query = """
CREATE TABLE IF NOT EXISTS `table_name` (
  vars
) ENGINE=InnoDB;
"""
cursor.execute(create_table4_query)


# show tables below
cursor.execute("SHOW TABLES")

# Fetch all the rows
tables = cursor.fetchall()

# Print the list of tables
for table in tables:
    print(table[0])

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()